In [1]:
pip install split-folders

In [10]:
!pip install gdown

# **Load DataSet**

Dataset: https://drive.google.com/file/d/1PFOrjqjpAAI9bT30QweynvjdDSV7MyaV/view?usp=sharing  
TestData: https://drive.google.com/drive/folders/182mqYOKJaSz9AjpFyJyPqw3oNQTln-pq?usp=sharing

In [25]:
!gdown https://drive.google.com/uc?id=1PFOrjqjpAAI9bT30QweynvjdDSV7MyaV


Downloading...
From (original): https://drive.google.com/uc?id=1PFOrjqjpAAI9bT30QweynvjdDSV7MyaV
From (redirected): https://drive.google.com/uc?id=1PFOrjqjpAAI9bT30QweynvjdDSV7MyaV&confirm=t&uuid=96ca6430-a80c-424b-891e-1a2c03c571c4
To: /content/archive_1.zip
100% 4.29G/4.29G [00:43<00:00, 98.7MB/s]


In [29]:
!gdown https://drive.google.com/uc?id=1A6Ds9Oka3O2EnhlRz-SNdjwU64Z-8QxS

Downloading...
From: https://drive.google.com/uc?id=1A6Ds9Oka3O2EnhlRz-SNdjwU64Z-8QxS
To: /content/Test_data.zip
100% 1.98M/1.98M [00:00<00:00, 109MB/s]


In [31]:
!unzip "/content/Test_data.zip"

Archive:  /content/Test_data.zip
  inflating: Test_data/6_testasl.jpg  
  inflating: Test_data/9_asl (1).jpg  
  inflating: Test_data/8(2)_asl.jpg  
  inflating: Test_data/5_asl.jpg     
  inflating: Test_data/7_asl.jpg     
  inflating: Test_data/0(2)_asl.jpg  
  inflating: Test_data/4_asl.jpg     
  inflating: Test_data/2_asl.jpg     
  inflating: Test_data/3_asl.jpg     
  inflating: Test_data/1_asl.jpg     


In [26]:
!unzip "/content/archive_1.zip"

Streaming output truncated to the last 5000 lines.
  inflating: asl_dataset/Z/Z (3168).jpg  
  inflating: asl_dataset/Z/Z (3169).jpg  
  inflating: asl_dataset/Z/Z (317).jpg  
  inflating: asl_dataset/Z/Z (3170).jpg  
  inflating: asl_dataset/Z/Z (3171).jpg  
  inflating: asl_dataset/Z/Z (3172).jpg  
  inflating: asl_dataset/Z/Z (3173).jpg  
  inflating: asl_dataset/Z/Z (3174).jpg  
  inflating: asl_dataset/Z/Z (3175).jpg  
  inflating: asl_dataset/Z/Z (3176).jpg  
  inflating: asl_dataset/Z/Z (3177).jpg  
  inflating: asl_dataset/Z/Z (3178).jpg  
  inflating: asl_dataset/Z/Z (3179).jpg  
  inflating: asl_dataset/Z/Z (318).jpg  
  inflating: asl_dataset/Z/Z (3180).jpg  
  inflating: asl_dataset/Z/Z (3181).jpg  
  inflating: asl_dataset/Z/Z (3182).jpg  
  inflating: asl_dataset/Z/Z (3183).jpg  
  inflating: asl_dataset/Z/Z (3184).jpg  
  inflating: asl_dataset/Z/Z (3185).jpg  
  inflating: asl_dataset/Z/Z (3186).jpg  
  inflating: asl_dataset/Z/Z (3187).jpg  
  inflating: asl_dataset/Z/

In [32]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import splitfolders

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D, BatchNormalization,Input,concatenate
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.utils import plot_model

from sklearn.metrics import classification_report, confusion_matrix

In [33]:
import os
import re
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split  # เพิ่ม import นี้เข้ามา

# Path where our data is located
base_path = "/content/asl_dataset"

# Dictionary to save our 10 classes (Update categories as needed)
categories = {i: str(i) for i in range(10)}

def add_class_name_prefix(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x[re.search("_", x).start() + 1: re.search("_", x).start() + 2] + "/" + x)
    return df

# Create dataframe with filenames and categories
filenames_list = []
categories_list = []

for category in categories:
    filenames = os.listdir(base_path + "/" + categories[category])
    filenames_list.extend(filenames)
    categories_list.extend([categories[category]] * len(filenames))

df = pd.DataFrame({"filename": filenames_list, "category": categories_list})
df = add_class_name_prefix(df, "filename").sample(frac=1).reset_index(drop=True)

# Split the dataframe into train, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

# Define target size
target_size = (64, 64)

# Image data generator with augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Image data generator for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the dataframe
def create_generator(datagen, dataframe, subset, target_size):
    return datagen.flow_from_dataframe(
        dataframe,
        base_path,
        x_col='filename',
        y_col='category',
        target_size=target_size,
        batch_size=32,
        class_mode='categorical',
        subset=subset,
        save_to_dir='/content/resized_dataset'  # กำหนดตำแหน่งที่จะบันทึก dataset ที่ปรับขนาดแล้ว
    )

train_generator = create_generator(train_datagen, train_df, None, target_size)
val_generator = create_generator(val_test_datagen, val_df, None, target_size)
test_generator = create_generator(val_test_datagen, test_df, None, target_size)


Found 415 validated image filenames belonging to 10 classes.
Found 130 validated image filenames belonging to 10 classes.
Found 150 validated image filenames belonging to 10 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 2372 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 799 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 779 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [36]:
splitfolders.ratio('/content/asl_dataset',output='/content/working/', seed=1333, ratio=(0.8,0.1,0.1))



Copying files: 9294 files [18:13,  8.50 files/s]  


Copying files: 1 files [00:00,  1.15 files/s]

Copying files: 286 files [00:00, 402.70 files/s]

Copying files: 633 files [00:01, 916.42 files/s]

Copying files: 966 files [00:01, 1382.32 files/s]

Copying files: 1327 files [00:01, 1863.61 files/s]

Copying files: 1689 files [00:01, 2277.89 files/s]

Copying files: 2037 files [00:01, 2580.13 files/s]

Copying files: 2403 files [00:01, 2864.52 files/s]

Copying files: 2748 files [00:01, 3022.56 files/s]

Copying files: 3093 files [00:01, 2237.14 files/s]

Copying files: 3375 files [00:02, 1851.14 files/s]

Copying files: 3609 files [00:02, 1682.20 files/s]

Copying files: 3812 files [00:02, 1592.86 files/s]

Copying files: 3994 files [00:02, 1531.85 files/s]

Copying files: 4162 files [00:02, 1482.30 files/s]

Copying files: 4320 files [00:02, 1493.92 files/s]

Copying files: 4480 files [00:02, 1518.67 files/s]

Copying files: 4638 files [00:03, 1469.34 files/s]

Copying files: 4818

In [ ]:
import shutil
shutil.rmtree("/content/working/train")
shutil.rmtree("/content/working/val")
shutil.rmtree("/content/working/test")

In [22]:
datagen = ImageDataGenerator(rescale= 1.0 / 255)

In [23]:
train_path = '/content/working/train'
val_path = '/content/working/val'
test_path = '/content/working/test'

batch = 32
image_size = 200
img_channel = 3
n_classes = 36

In [ ]:
model = Sequential()
# input layer
# Block 1
model.add(Conv2D(32,3,activation='relu',padding='same',input_shape = (image_size,image_size,img_channel)))
model.add(Conv2D(32,3,activation='relu',padding='same'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.2))

# Block 2
model.add(Conv2D(64,3,activation='relu',padding='same'))
model.add(Conv2D(64,3,activation='relu',padding='same'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.3))

#Block 3
model.add(Conv2D(128,3,activation='relu',padding='same'))
model.add(Conv2D(128,3,activation='relu',padding='same'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.4))

# fully connected layer
model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(36, activation='softmax'))



model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 200, 200, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 100, 100, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 64)      36928     
                                                        

---------
\\

---

**`Run`**

---


---



#model 2

In [57]:
import os
import re
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Path where our data is located
base_path = "/content/asl_dataset"

# Dictionary to save our 10 classes (Update categories as needed)
categories = {i: str(i) for i in range(10)}

def add_class_name_prefix(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x[re.search("_", x).start() + 1: re.search("_", x).start() + 2] + "/" + x)
    return df

# Create dataframe with filenames and categories
filenames_list = []
categories_list = []

for category in categories:
    filenames = os.listdir(base_path + "/" + categories[category])
    filenames_list.extend(filenames)
    categories_list.extend([categories[category]] * len(filenames))

df = pd.DataFrame({"filename": filenames_list, "category": categories_list})
df = add_class_name_prefix(df, "filename").sample(frac=1).reset_index(drop=True)

# Split the dataframe into train, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

# Image data generator with augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

# Image data generator for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the dataframe
def create_generator(datagen, dataframe, subset):
    return datagen.flow_from_dataframe(
        dataframe,
        base_path,
        x_col='filename',
        y_col='category',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        subset=subset
    )

train_generator = create_generator(train_datagen, train_df, None)
val_generator = create_generator(val_test_datagen, val_df, None)
test_generator = create_generator(val_test_datagen, test_df, None)

# Define the model
model = Sequential()
# input layer
# Block 1
model.add(Conv2D(32, 3, activation='relu', padding='same', input_shape=(64, 64, 3)))
model.add(Conv2D(32, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.2))

# Block 2
model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.3))

# Block 3
model.add(Conv2D(128, 3, activation='relu', padding='same'))
model.add(Conv2D(128, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.4))

# fully connected layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(10, activation='softmax'))  # ต้องการให้ผลลัพธ์มี 10 คลาสตามจำนวนของ categories ที่มี

# Compile the model with Adam optimizer and a learning rate of 0.0001
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model without the ReduceLROnPlateau callback
asl_class = model.fit(train_generator, validation_data=val_generator, epochs=100, verbose=2)

# Evaluate on training data
loss, acc = model.evaluate(train_generator, verbose=0)
print('The accuracy of the model for training data is:', acc * 100)
print('The Loss of the model for training data is:', loss)

# Evaluate on validation data
loss, acc = model.evaluate(val_generator, verbose=0)
print('The accuracy of the model for validation data is:', acc * 100)
print('The Loss of the model for validation data is:', loss)

# Prediction
result = model.predict(test_generator, verbose=0)
y_pred = np.argmax(result, axis=1)
y_true = test_generator.classes

# Evaluate on test data
loss, acc = model.evaluate(test_generator, verbose=0)
print('The accuracy of the model for testing data is:', acc * 100)
print('The Loss of the model for testing data is:', loss)

# Calculate correct and incorrect predictions
correct = np.nonzero(y_pred == y_true)[0]
incorrect = np.nonzero(y_pred != y_true)[0]
print("Correct predicted classes:", correct.shape[0])
print("Incorrect predicted classes:", incorrect.shape[0])

# Save the model
model_path = "CNN_LongDay.h5"
model.save(model_path)
print(f"Model saved to '{model_path}'")


Found 419 validated image filenames belonging to 10 classes.
Found 138 validated image filenames belonging to 10 classes.
Found 138 validated image filenames belonging to 10 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 27 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 11 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 11 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/100
14/14 - 19s - loss: 2.3069 - accuracy: 0.1265 - val_loss: 2.3094 - val_accuracy: 0.1014 - 19s/epoch - 1s/step
Epoch 2/100
14/14 - 15s - loss: 2.2995 - accuracy: 0.0979 - val_loss: 2.3114 - val_accuracy: 0.1014 - 15s/epoch - 1s/step
Epoch 3/100
14/14 - 15s - loss: 2.2974 - accuracy: 0.1050 - val_loss: 2.3173 - val_accuracy: 0.0797 - 15s/epoch - 1s/step
Epoch 4/100
14/14 - 15s - loss: 2.2996 - accuracy: 0.1146 - val_loss: 2.3071 - val_accuracy: 0.1304 - 15s/epoch - 1s/step
Epoch 5/100
14/14 - 15s - loss: 2.3013 - accuracy: 0.1050 - val_loss: 2.3075 - val_accuracy: 0.0797 - 15s/epoch - 1s/step
Epoch 6/100
14/14 - 16s - loss: 2.3003 - accuracy: 0.1098 - val_loss: 2.3135 - val_accuracy: 0.0797 - 16s/epoch - 1s/step
Epoch 7/100
14/14 - 15s - loss: 2.2978 - accuracy: 0.1146 - val_loss: 2.3160 - val_accuracy: 0.0797 - 15s/epoch - 1s/step
Epoch 8/100
14/14 - 15s - loss: 2.2963 - accuracy: 0.1050 - val_loss: 2.3045 - val_accuracy: 0.0797 - 15s/epoch - 1s/step
Epoch 9/100
14/14 - 15s 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to 'CNN_LongDay.h5'


In [98]:
import requests

def remove_background(image_path, api_key):
    response = requests.post(
        'https://api.remove.bg/v1.0/removebg',
        files={'image_file': open(image_path, 'rb')},
        data={'size': 'auto'},
        headers={'X-Api-Key': api_key},
    )
    if response.status_code == requests.codes.ok:
        with open('no_background.png', 'wb') as out:
            out.write(response.content)
        print("Background removed successfully!")
    else:
        print("Error:", response.status_code, response.text)

# ใส่ API Key ที่ได้จาก remove.bg ที่นี่
api_key = 'dnctVUfZ1scLQDPNpknq5kvs'

# ใส่ path ของรูปภาพที่ต้องการลบพื้นหลังที่นี่
image_path = '/content/23932_0.jpg'

# เรียกใช้ฟังก์ชันเพื่อลบพื้นหลังของรูปภาพ
remove_background(image_path, api_key)


Error: 402 {"errors":[{"title":"Insufficient credits","code":"insufficient_credits"}]}


In [102]:
import requests

def remove_background(image_path, api_key):
    try:
        with open(image_path, 'rb') as image_file:
            response = requests.post(
                'https://api.remove.bg/v1.0/removebg',
                files={'image_file': image_file},
                data={'size': 'auto'},
                headers={'X-Api-Key': api_key},
            )

        if response.status_code == requests.codes.ok:
            with open('no_background.png', 'wb') as out:
                out.write(response.content)
            print("Background removed successfully!")
        elif response.status_code == 402:
            print("Error: Insufficient credits. Please check your API key or purchase more credits.")
        else:
            print(f"Error: {response.status_code} - {response.text}")

    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Insert your API Key from remove.bg here
api_key = 'sRZ3cJSkMdz77kPgUBzrNpsS'

# Insert the path of the image you want to remove the background from here
image_path = '/content/23932_0.jpg'

# Call the function to remove the background of the image
remove_background(image_path, api_key)


Error: Insufficient credits. Please check your API key or purchase more credits.


In [99]:
import numpy as np
import cv2
import tensorflow as tf

model_path = "/content/CNN_LongDay.h5"
try:
    model = tf.keras.models.load_model(model_path)
except OSError as e:
    print(f"Error loading model from '{model_path}': {e}")
    exit()

background = None
accumulated_weight = 0.5
ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350
letters = {
    0: "0",
    1: "1",
    2: "2",
    3: "3",
    4: "4",
    5: "5",
    6: "6",
    7: "7",
    8: "8",
    9: "9",
    10: "A",
    11: "B",
    12: "C",
    13: "D",
    14: "E",
    15: "F",
    16: "G",
    17: "H",
    18: "I",
    19: "J",
    20: "K",
    21: "L",
    22: "M",
    23: "N",
    24: "O",
    25: "P",
    26: "Q",
    27: "R",
    28: "S",
    29: "T",
    30: "U",
    31: "V",
    32: "W",
    33: "X",
    34: "Y",
    35: "Z"
}

# Load an image for testing
image_path = "/content/no_background.png"  # Provide the path to your test image
test_image = cv2.imread(image_path)
test_image = cv2.flip(test_image, 1)  # Flip horizontally if needed

# Preprocess the image
gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (9, 9), 0)

# Resize the image to match the model's input shape (64x64) and convert to three channels
input_size = (64, 64)
resized_image = cv2.resize(gray, input_size)
resized_image = cv2.merge([resized_image]*3)  # Duplicate the single channel to create three channels
resized_image = resized_image.reshape(-1, input_size[0], input_size[1], 3) / 255.0  # Normalize

prediction = model.predict(resized_image)
predicted_letter_index = np.argmax(prediction)
predicted_letter = letters[predicted_letter_index]

print("Predicted Letter:", predicted_letter)


1/1 [==============================] - 0s 148ms/step
Predicted Letter: 9
